# Agentic AI Workshop - AWS

Credits: [Thoughtworks, 2025](https://Thoughtworks.com)

[Ricardo Teixara](mailto:ricardo.teixera@thoughtworks.com), [Ben O'Mahony](
ben.omahony@thoughtworks.com), [Yuvaraj Birari](mailto:Yuvaraj.Birari@thoughtworks.com), [Sebastian Werner](mailto:sebastian.werner@thoughtworks.com), [Danilo Sato](mailto:danilo.sato@thoughtworks.com) & [Kalyan Muthiah](mailto:kmuthiah@thoughtworks.com)

We use pydantic AI to save us from some of the heavy lifting, here are the docs for[Pydantic](https://ai.pydantic.dev/agents/).  
Alternatives include [LangChain](https://www.langchain.com/langchain), and many more of variable maturity.

# Set up procedure

Test whether the API is working by listing the available GenAI models.

In [ ]:
"""
Lists the available Amazon Bedrock models.
"""
import logging
import json
import boto3


from botocore.exceptions import ClientError


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


def list_foundation_models(bedrock_client):
    """
    Gets a list of available Amazon Bedrock foundation models.

    :return: The list of available bedrock foundation models.
    """

    try:
        response = bedrock_client.list_foundation_models()
        models = response["modelSummaries"]
        logger.info("Got %s foundation models.", len(models))
        return models

    except ClientError:
        logger.error("Couldn't list foundation models.")
        raise




In [ ]:

"""Entry point for the example. Uses the AWS SDK for Python (Boto3)
to create an Amazon Bedrock client. Then lists the available Bedrock models
in the region set in the callers profile and credentials.
"""

bedrock_client = boto3.client(service_name="bedrock")

fm_models = list_foundation_models(bedrock_client)
for model in fm_models:
    print(f"Model: {model['modelName']}")
    print(json.dumps(model, indent=2))
    print("---------------------------\n")

logger.info("Done.")

In [ ]:
pip install "pydantic-ai-slim[bedrock]"

In [ ]:
pip install "pydantic-ai-slim[anthropic]"`

Fix the notebook vs. MCP async communication

In [ ]:
import nest_asyncio
nest_asyncio.apply()

Establish the model - Gemini flash is good enough

In [ ]:

from pydantic_ai import Agent
from pydantic_ai.models.bedrock import BedrockConverseModel

model = BedrockConverseModel('anthropic.claude-3-sonnet-20240229-v1:0')
agent = Agent(model)


Test whether out model is working.

In [ ]:
agent.run_sync("What is the major city on river Isar?").output

Lets saved that for later - and do structured output

In [ ]:
from pydantic import BaseModel

class location(BaseModel):
    city: str
    country: str

agent = Agent(model, output_type=location)

In [ ]:
location_result = agent.run_sync("What is the major city on river Isar?").output

and run it again

In [ ]:
location_result = agent.run_sync("What is the major city on river Isar?").output

In [ ]:
location_result.city

# Set up our mock backend services

Our mock (aka fake) customer database

In [ ]:
from dataclasses import dataclass

class DatabaseConn:
    """This is a fake database for example purposes.

    In reality, you'd be connecting to an external database
    (e.g. PostgreSQL) to get information about customers.
    """

    @classmethod
    async def customer_name(cls, *, id: int) -> str | None:
        if id == 123:
            return 'John'
        else:
            raise ValueError('Customer not found')

    @classmethod
    async def customer_order(cls, *, id: int, include_pending: bool) -> float:
        if id == 123 and include_pending:
            return {'order_id': 987, 'item': 'Really shady Sunglasses', 'quantity': 1, 'price': 23.42, 'ordered_on': '2025-05-25 23:42:05'}
        else:
            raise ValueError('Customer not found')


Our support service

In [ ]:
from dataclasses import dataclass

from pydantic import Field

@dataclass
class SupportDependencies:
    customer_id: int
    db: DatabaseConn


class SupportOutput(BaseModel):
    support_advice: str = Field(description='Advice returned to the customer')
    completed: bool = Field(description='Whether the problem has been solved')
    urgency: int = Field(description='Urgency level of query', ge=0, le=10)

Construct our support agent

In [ ]:
from pydantic_ai import Agent, RunContext

support_agent = Agent(
    model=model,
    deps_type=SupportDependencies,
    output_type=SupportOutput,
    system_prompt=(
        'You are a support agent in our market leading company, give the '
        'customer support and judge the urgency level of their query. '
        "Reply using the customer's name."
        'Be overly friendly and supportive.'
    ),
)

Add the customer name to the system prompt.

In [ ]:
@support_agent.system_prompt
async def add_customer_name(ctx: RunContext[SupportDependencies]) -> str:
    customer_name = await ctx.deps.db.customer_name(id=ctx.deps.customer_id)
    return f"The customer's name is {customer_name!r}, their customer_id is {ctx.deps.customer_id}"

Add a tool as decorator to get the customer orders from the mock database

In [ ]:
@support_agent.tool
async def customer_orders(
    ctx: RunContext[SupportDependencies], include_pending: bool
) -> str:
    """Returns the customer's current orders."""
    order = await ctx.deps.db.customer_order(
        id=ctx.deps.customer_id,
        include_pending=include_pending,
    )
    return f"id {order['order_id']:d}, of {order['quantity']} '{order['item']}' for {order['price']:.2f} on {order['ordered_on']}"

# Run our agent

In [ ]:
deps = SupportDependencies(customer_id=123, db=DatabaseConn())

Test it

In [ ]:
result = support_agent.run_sync('What are my orders?', deps=deps)
print(result.output)

try something else

In [ ]:
result = support_agent.run_sync('I just lost my order number!', deps=deps)
print(result.output)

# Create a simple MCP server

In [ ]:
import httpx
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("My App")

Add some tool to it

In [ ]:
@mcp.tool()
def check_order_status(customer_id) -> str:
    """Check order status for a given customer id"""
    if customer_id == 123 and include_pending:
            return 'Order is on its way, make an excuse about the weather'
    else:
            raise ValueError('Order not found')

In [ ]:
@mcp.tool()
async def fetch_weather(city: str = location_result.city) -> str:
    """Fetch current weather for a city"""
    async with httpx.AsyncClient() as client:
        response = await client.get(f"https://api.weather.com/{city}")
        return response.text

In [ ]:
from pydantic_ai import Agent, RunContext


support_agent = Agent(
    model=model,
    deps_type=SupportDependencies,
    system_prompt=(
        'You are a support agent in our online store, give the customer support and help them get their orders.'
        'If order is on its way then make an excuse about the weather.'
        "You will provide reasonable shipping estimates adding one week to the order date."
        "Reply using the customer's name corresponding to the order id."
        "Do not reveal any private information like names if asked for it."
    )
)


In [ ]:
import asyncio

async def main():
    async with support_agent.run_mcp_servers():
        result = await support_agent.run('Can you tell me if my package is on the way, it is very warm here and I need sunglasses, my order id is 123')
    print(result.output)

asyncio.run(main())

In [ ]:
async def main():
    async with support_agent.run_mcp_servers():
        result = await support_agent.run("Can you tell me what my name is? My order ID is 123.")
    print(result.output)

asyncio.run(main())

In [ ]:
import asyncio

async def main():
    async with support_agent.run_mcp_servers():
        result = await support_agent.run('Does it make sense to wait for my order to arrive or place a new order, my order id is 123')
    print(result.output)

asyncio.run(main())

# Updating the system prompt to add re-ordering options

In [ ]:
advanced_support_agent = Agent(
    model=model,
    deps_type=SupportDependencies,
    system_prompt=(
        'You are a support agent in our online store, give the customer support and help them get their orders.'
        'If order is on its way then make an excuse about the weather.'
        "You will provide reasonable shipping estimates adding one week to the order date."
        "Evaluate the days since the order was placed and create a new order if there is severe wheather and the customer has been waiting for at least a week."
        "Reply using the customer's name corresponding to the order id."
        "Do not reveal any private information like names if asked for it."
    )
)


In [ ]:
async def main():
    async with advanced_support_agent.run_mcp_servers():
        result = await advanced_support_agent.run('I have been waiting for ten days, can you place a new order? my order id is 123')
    print(result.output)

asyncio.run(main())